In [2]:
!pip install tensorflow opencv-python mediapipe

INFO: pip is looking at multiple versions of tensorboard to determine which version is compatible with other requirements. This could take a while.
  Obtaining dependency information for tensorboard<2.16,>=2.15 from https://files.pythonhosted.org/packages/37/12/f6e9b9dcc310263cbd3948274e286538bd6800fd0c268850788f14a0c6d0/tensorboard-2.15.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/5.5 MB 217.9 kB/s eta 0:00:26
    --------------------------------------- 0.1/5.5 MB 762.6 kB/s eta 0:00:08
   - -------------------------------------- 0.2/5.5 MB 1.2 MB/s eta 0:00:05
   - -------------------------------------- 0.2/5.5 MB 1.1 MB/s eta 0:00:05
   -- ------------------------------------- 0.3/5.5 MB 1.3 MB/s eta 0:00:05
   -- ------------------------------------- 0.4/5.5 MB 1.4 MB/s eta 0:00:04
   --- ---------------------------

In [1]:
import cvzone
import mediapipe
import cv2

In [ ]:
import cv2
import numpy as np

# Initialize video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Define keys and keyboard layout
keys = [
    ['Q', 'W', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P'],
    ['A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'L', ';'],
    ['Z', 'X', 'C', 'V', 'B', 'N', 'M', ',', '.', '/']
]
button_width, button_height = 80, 80
keyboard_x, keyboard_y = 100, 100

# Function to draw the keyboard
def draw_keyboard(img):
    for row_idx, row in enumerate(keys):
        for key_idx, key in enumerate(row):
            x = keyboard_x + key_idx * button_width
            y = keyboard_y + row_idx * button_height
            cv2.rectangle(img, (x, y), (x + button_width, y + button_height), (200, 200, 200), cv2.FILLED)
            cv2.rectangle(img, (x, y), (x + button_width, y + button_height), (0, 0, 0), 2)
            cv2.putText(img, key, (x + 20, y + 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

# Mouse callback function
def on_touch(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        for row_idx, row in enumerate(keys):
            for key_idx, key in enumerate(row):
                x1 = keyboard_x + key_idx * button_width
                y1 = keyboard_y + row_idx * button_height
                x2 = x1 + button_width
                y2 = y1 + button_height
                if x1 < x < x2 and y1 < y < y2:
                    print(f'Key pressed: {key}')

# Set mouse callback
cv2.namedWindow("Virtual Keyboard")
cv2.setMouseCallback("Virtual Keyboard", on_touch)

# Main loop
while True:
    success, img = cap.read()
    if not success:
        break
    draw_keyboard(img)
    cv2.imshow("Virtual Keyboard", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Key pressed: M
Key pressed: L
Key pressed: A
Key pressed: M
Key pressed: E


In [ ]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import tensorflow as tf
from tensorflow.compat.v1.losses import sparse_softmax_cross_entropy
import numpy as np
import time

# Ensure compatibility with TensorFlow v1 deprecated functions
tf.compat.v1.disable_eager_execution()

# Initialize the video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # Set width
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)  # Set height

# Initialize the hand detector
detector = HandDetector(detectionCon=0.8, maxHands=1)

# List of keys
keys = [
    ['Q', 'W', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P'],
    ['A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'L', ';'],
    ['Z', 'X', 'C', 'V', 'B', 'N', 'M', ',', '.', '/']
]

# Keyboard layout dimensions
button_width, button_height = 80, 80
keyboard_x, keyboard_y = 100, 100  # Starting position of keyboard

# Function to draw the virtual keyboard
def draw_keyboard(img, keys, button_width, button_height, keyboard_x, keyboard_y):
    for row_idx, row in enumerate(keys):
        for key_idx, key in enumerate(row):
            x = keyboard_x + key_idx * button_width
            y = keyboard_y + row_idx * button_height
            cv2.rectangle(img, (x, y), (x + button_width, y + button_height), (255, 0, 0), cv2.FILLED)
            cv2.putText(img, key, (x + 20, y + 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Function to detect which key is being pressed
def detect_key_pressed(lmList, keys, button_width, button_height, keyboard_x, keyboard_y):
    if lmList:
        for row_idx, row in enumerate(keys):
            for key_idx, key in enumerate(row):
                x = keyboard_x + key_idx * button_width
                y = keyboard_y + row_idx * button_height
                # Check if fingertip is within the button's area
                if x < lmList[8][0] < x + button_width and y < lmList[8][1] < y + button_height:
                    cv2.rectangle(img, (x, y), (x + button_width, y + button_height), (0, 255, 0), cv2.FILLED)
                    cv2.putText(img, key, (x + 20, y + 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    return key
    return None

# Main loop
while True:
    success, img = cap.read()
    if not success:
        break  # Exit the loop if the frame was not captured successfully
    
    # Detect hands in the image
    hands, img = detector.findHands(img)
    
    # Get the list of landmarks and bounding box info
    if hands:
        lmList = hands[0]['lmList']  # List of 21 Landmark points
        bbox = hands[0]['bbox']  # Bounding box info x, y, w, h
        centerPoint = hands[0]['center']  # center of the hand cx, cy
        handType = hands[0]['type']  # Hand type Left or Right
    
    # Draw the keyboard
    draw_keyboard(img, keys, button_width, button_height, keyboard_x, keyboard_y)
    
    # Detect key pressed
    key = detect_key_pressed(lmList if hands else [], keys, button_width, button_height, keyboard_x, keyboard_y)
    
    if key:
        print(f'Key pressed: {key}')
    
    # Display the image
    cv2.imshow("Virtual Keyboard", img)
    
    # Exit loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


C:\Users\swaminarayan\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Key pressed: /
Key pressed: L
Key pressed: L
Key pressed: L
Key pressed: L
Key pressed: L
Key pressed: O
Key pressed: P
Key pressed: P
Key pressed: P
Key pressed: P
Key pressed: ;
Key pressed: ;
Key pressed: ;
Key pressed: /
Key pressed: /
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: .
Key presse

In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np

# Initialize the video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)  # Set width
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)  # Set height

# Initialize the hand detector
detector = HandDetector(detectionCon=0.8, maxHands=1)

# List of keys
keys = [
    ['Q', 'W', 'E', 'R', 'T', 'Y', 'U', 'I', 'O', 'P'],
    ['A', 'S', 'D', 'F', 'G', 'H', 'J', 'K', 'L', ';'],
    ['Z', 'X', 'C', 'V', 'B', 'N', 'M', ',', '.', '/']
]

# Keyboard layout dimensions
button_width, button_height = 80, 80
keyboard_x, keyboard_y = 100, 100  # Starting position of keyboard

# Function to draw the virtual keyboard
def draw_keyboard(img, keys, button_width, button_height, keyboard_x, keyboard_y):
    for row_idx, row in enumerate(keys):
        for key_idx, key in enumerate(row):
            x = keyboard_x + key_idx * button_width
            y = keyboard_y + row_idx * button_height
            cv2.rectangle(img, (x, y), (x + button_width, y + button_height), (255, 0, 0), cv2.FILLED)
            cv2.putText(img, key, (x + 20, y + 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

# Function to detect which key is being pressed
def detect_key_pressed(lmList, keys, button_width, button_height, keyboard_x, keyboard_y):
    if lmList:
        for row_idx, row in enumerate(keys):
            for key_idx, key in enumerate(row):
                x = keyboard_x + key_idx * button_width
                y = keyboard_y + row_idx * button_height
                # Check if fingertip (landmark index 8) is within the button's area
                if x < lmList[8][0] < x + button_width and y < lmList[8][1] < y + button_height:
                    cv2.rectangle(img, (x, y), (x + button_width, y + button_height), (0, 255, 0), cv2.FILLED)
                    cv2.putText(img, key, (x + 20, y + 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    return key
    return None

# Gesture detection function (checks if index finger is bent)
def detect_click_gesture(lmList):
    if lmList:
        # Landmark indices: 6 (base of index finger), 8 (tip of index finger), 10 (middle joint of index finger)
        # Calculate angle between these points
        if len(lmList) >= 11:  # Ensure enough landmarks are detected
            finger_base = lmList[6]
            finger_tip = lmList[8]
            finger_middle = lmList[10]
            # Calculate angle using vector mathematics
            v1 = np.array([finger_base[0] - finger_middle[0], finger_base[1] - finger_middle[1]])
            v2 = np.array([finger_tip[0] - finger_middle[0], finger_tip[1] - finger_middle[1]])
            angle = np.degrees(np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))))
            # Angle threshold to detect finger bend (adjust as needed)
            angle_threshold = 100
            if angle < angle_threshold:
                return True
    return False

# Main loop
while True:
    success, img = cap.read()
    if not success:
        break  # Exit the loop if the frame was not captured successfully
    
    # Detect hands in the image
    hands, img = detector.findHands(img)
    
    # Get the list of landmarks and bounding box info
    lmList = None
    if hands:
        lmList = hands[0]['lmList']  # List of 21 Landmark points
        bbox = hands[0]['bbox']  # Bounding box info x, y, w, h
        centerPoint = hands[0]['center']  # center of the hand cx, cy
        handType = hands[0]['type']  # Hand type Left or Right
    
    # Draw the keyboard
    draw_keyboard(img, keys, button_width, button_height, keyboard_x, keyboard_y)
    
    # Detect key pressed
    key = detect_key_pressed(lmList, keys, button_width, button_height, keyboard_x, keyboard_y)
    
    # Detect click gesture and update output accordingly
    if detect_click_gesture(lmList):
        if key:
            print(f'Key pressed: {key}')
            # Here you can append the pressed key to a string or perform other actions
            # For example, append to a text string or perform some action based on the key
    
    # Display the image
    cv2.imshow("Virtual Keyboard", img)
    
    # Exit loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


C:\Users\swaminarayan\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Key pressed: ,
Key pressed: ,
Key pressed: K
Key pressed: K
Key pressed: L
Key pressed: L
Key pressed: L
Key pressed: L
Key pressed: L
Key pressed: L
Key pressed: .
Key pressed: .
Key pressed: .
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key pressed: ,
Key presse